<a href="https://colab.research.google.com/github/swaminaathakrishnan/Cool_Route_prototype/blob/master/notebooks/Cool_route_v10.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 **CoolRide V10.2: The "Production-Grade" Engine**
### *Live AI. Anti-Throttle Failover. Satellite Intelligence.*

---

### **🌍 Project Overview**
CoolRide V10.2 is the fully realized **"Digital Twin"** of Singapore's urban heat profile. It combines real-time shadow simulation, tree canopy analysis, and blue infrastructure cooling to find the safest, coolest path for cyclists.

Unlike early prototypes, V10.2 is a **Robust Distributed System**. It runs heavy AI computation on a cloud GPU/CPU (Colab), exposes a secure API tunnel (Ngrok), and serves a responsive, multi-language web application to any device instantly.

---

### **🔥 What's New in V10.2? (The "Robustness" Update)**

1.  **🛡️ Anti-Throttle "Failover" Engine:**
    * **Problem:** Heavy map downloads (e.g., cross-island routes) can trigger server bans.
    * **Solution:** The engine now automatically detects bans and instantly switches to backup mirrors (e.g., `kumi.systems` or `openstreetmap.fr`) to ensure zero downtime during demos.

2.  **🛰️ Satellite Intelligence & Smart Metrics:**
    * **Visuals:** Users can toggle between **Street Maps** and **Esri Satellite Imagery** for realistic route planning.
    * **Metrics:** The AI now calculates **"Shade Gain"** (e.g., *"25% more shade coverage"*), proving the value of the cool route over the fastest route.

3.  **🤖 Physics-Informed AI Forecasting:**
    * Uses a **Diurnal Cycle Model** (Sine Wave Regression) to predict heat stress 1 hour into the future based on the sun's position and current sensor data.

4.  **🌐 Global Accessibility Suite:**
    * **Multi-Language:** Instantly toggle between **English**, **Mandarin (中文)**, and **Tamil (தமிழ்)**.
    * **Interactivity:** Click on **Hawker Centres (🍜)** or **Supermarkets (🛒)** to instantly add a "Pit Stop."
    * **Data Export:** Download routes as `.kml` for Google Earth.

---

### **🏗️ System Architecture**

`[ 📱 Web App (Frontend) ]`  <--->  `[ 🚇 Ngrok Secure Tunnel ]`  <--->  `[ 🧠 Python AI Server (Backend) ]`

1.  **Request:** User selects Start/End on the website.
2.  **Failover Check:** Engine attempts map download from Main Server -> Backup Mirror 1 -> Backup Mirror 2.
3.  **Compute:** Python Engine calculates Shadows, Trees, Water, and AI Weather trends.
4.  **Render:** The Web App draws the route, amenities, and AI metrics card in < 200ms.

---

### **🏃 How to Run the Demo (Live Protocol)**

**Step 1: Launch the Brain**
* In this notebook, click **Runtime -> Run All**.
* Scroll to the bottom of **Module 7**.
* Copy the public URL: `https://xxxx-xxxx.ngrok-free.app`

**Step 2: Connect the Interface**
* Open the [Live Dashboard](https://swaminaathakrishnan.github.io/Cool_Route_prototype/).
* Paste the URL into the **"Server Connection"** box.

**Step 3: The "Safe Demo" Strategy (CRITICAL)**
* **Start Small:** Search **"Tampines MRT"** to **"Tampines Eco Green"**. This loads in <10s and proves the system works.
* **Show Features:** Toggle **Satellite Mode**, add a **Hawker Stop**, and explain the **"Shade Gain"** metric.
* **⚠️ Safety Warning:** Avoid running "Changi to Jurong" (35km) live. Massive downloads take ~60s and risk hitting API rate limits. Stick to district-level routes (e.g., Bedok, Marina Bay) for speed.

---

### **👥 Credits**
* **Swaminaatha Krishnan:** System Architect & Full-Stack Integration
* **Arishya Jindal:** Algorithm Lead (Shadows & Spatial Intelligence)

In [1]:
# ==========================================
# 🧱 MODULE 1: SYSTEM INITIALIZATION
# ==========================================
# Objective: Install geospatial libraries and set up the environment.
# dependencies: OSMnx (Maps), GeoPandas (Spatial Data), Scikit-Learn (AI).

!pip install osmnx simplekml geopandas shapely networkx requests scikit-learn -q flask_cors pyngrok pysolar

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import os
import pickle
import time
from shapely.geometry import Point, LineString, box
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

print("✅ System Initialized. Ready for V5 Execution.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.2 MB/s eta 0:00:00
✅ System Initialized. Ready for V5 Execution.


### ⚙️ Module 2: Configuration & Cloud Connection
Objective: Define the pilot zone and connect to the GitHub Data Lake. Logic: Instead of local files, we stream GeoJSON/CSV directly from the raw GitHub URLs. This allows the team to collaborate without sharing Drive folders.

In [2]:
# ==========================================
# ⚙️ MODULE 2: CONFIGURATION (V5.3 - AUTOMATION READY)
# ==========================================
import pytz
from datetime import datetime

print("🚴 COOLRIDE V5.3 - PRODUCTION ENGINE")
print("=" * 50)

# 1. ROUTE PARAMETERS (Bridge Variables)
# Default: Tampines Loop
START_NAME = "Tampines MRT"
START_COORDS = (1.3533, 103.9452)

END_NAME = "Tampines Eco Green"
END_COORDS = (1.3598, 103.9351)

# 2. TIME CONFIGURATION (CRITICAL FIX: UTC+8)
# We force the timezone to Asia/Singapore so shadows are accurate.
sgt_zone = pytz.timezone('Asia/Singapore')
current_time_sgt = datetime.now(sgt_zone)

# Manual Override for Demo (Optional)
# DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0)

# Force 2:00 PM today for shadow simulation
DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0, second=0)
# DEPARTURE_TIME = current_time_sgt

PLACE_NAME = f"{START_NAME} to {END_NAME}"
START_POINT = START_COORDS
END_POINT = END_COORDS

print(f"   📍 Route: {PLACE_NAME}")
print(f"   🕐 Departure Time (SGT): {DEPARTURE_TIME.strftime('%I:%M %p')}")
print(f"      (Sun position calculated for: {DEPARTURE_TIME})")

# 3. GITHUB DATA LAKE
GITHUB_USER = "swaminaathakrishnan"
REPO_NAME = "Cool_Route_prototype"
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/"

# File Links
PCN_URL = BASE_URL + "ParkConnectorLoop.geojson"
HAWKER_URL = BASE_URL + "HawkerCentresGEOJSON.geojson"
TREES_URL = BASE_URL + "trees.csv"
URA_WATER_URL = BASE_URL + "URA_Waterbody.geojson"

# AMENITIES
HAWKER_URL = BASE_URL + "hawker_centres.geojson"
SUPERMARKET_URL = BASE_URL + "supermarkets.geojson"

# 4. THERMAL WEIGHTS
WEIGHT_PCN = 0.5
WEIGHT_WATER = 0.55
WEIGHT_TREE_SHADE = 0.6
WEIGHT_BUILDING_SHADE = 0.7
WEIGHT_ULTIMATE = 0.35 # All factors combined

# 5. WATER PARAMETERS
WATER_BUFFER_DISTANCE = 100
MIN_WATER_SIZE = 50000

# 6. SAFETY OVERRIDE
NEA_HEATWAVE_ALERT = False

print("=" * 50)

🚴 COOLRIDE V5.3 - PRODUCTION ENGINE
   📍 Route: Tampines MRT to Tampines Eco Green
   🕐 Departure Time (SGT): 02:00 PM
      (Sun position calculated for: 2025-12-24 14:00:00.613424+08:00)


### Module 3A: Sun Position & Building Shadow Engine


Objective: Calculate real-time building shadows based on sun position and building heights.

**Physics:**
- Sun elevation angle determines shadow length
- Shadow direction is opposite of sun azimuth
- Taller buildings cast longer shadows (especially at low sun angles)

In [3]:

# MODULE 3A: SUN POSITION & BUILDING SHADOWS (V4)

def calculate_sun_position(latitude, longitude, timestamp):
    """
    Calculate sun elevation and azimuth for given location and time

    Returns: (elevation_degrees, azimuth_degrees)
    """
    import math
    day_of_year = timestamp.timetuple().tm_yday

    # Declination angle
    declination = 23.45 * math.sin(math.radians((360/365) * (day_of_year - 81)))

    # Hour angle
    hour = timestamp.hour + timestamp.minute / 60.0
    hour_angle = 15 * (hour - 12)

    # Sun elevation
    lat_rad = math.radians(latitude)
    dec_rad = math.radians(declination)
    ha_rad = math.radians(hour_angle)

    sin_elev = (math.sin(lat_rad) * math.sin(dec_rad) +
                math.cos(lat_rad) * math.cos(dec_rad) * math.cos(ha_rad))
    elevation = math.degrees(math.asin(max(-1, min(1, sin_elev))))

    # Sun azimuth
    cos_azim = ((math.sin(dec_rad) - math.sin(lat_rad) * sin_elev) /
                (math.cos(lat_rad) * math.cos(math.radians(elevation))))
    cos_azim = max(-1, min(1, cos_azim))
    azimuth = math.degrees(math.acos(cos_azim))

    if hour > 12:
        azimuth = 360 - azimuth

    return elevation, azimuth


def create_shadow_polygon(building_polygon, building_height, sun_elevation, sun_azimuth):
    """
    Create shadow polygon from building footprint

    Args:
        building_polygon: Shapely Polygon
        building_height: Height in meters
        sun_elevation: Sun angle above horizon (degrees)
        sun_azimuth: Sun compass direction (degrees)

    Returns:
        Shadow polygon (Shapely)
    """
    import math
    from shapely.affinity import translate

    if sun_elevation <= 0:
        return None  # Night time

    # Shadow length = height / tan(elevation)
    shadow_length = building_height / math.tan(math.radians(sun_elevation))

    # Shadow direction (opposite of sun)
    shadow_direction = (sun_azimuth + 180) % 360

    # Calculate offset in meters
    shadow_offset_y = shadow_length * math.cos(math.radians(shadow_direction))
    shadow_offset_x = shadow_length * math.sin(math.radians(shadow_direction))

    # Get building centroid
    centroid = building_polygon.centroid
    lat, lon = centroid.y, centroid.x

    # Convert meters to degrees
    deg_per_meter_lat = 1 / 111000
    deg_per_meter_lon = 1 / (111000 * math.cos(math.radians(lat)))

    offset_lat = shadow_offset_y * deg_per_meter_lat
    offset_lon = shadow_offset_x * deg_per_meter_lon

    # Create shadow by translating building polygon
    shadow = translate(building_polygon, xoff=offset_lon, yoff=offset_lat)

    # Union with building for full coverage
    full_shadow = building_polygon.union(shadow).convex_hull

    return full_shadow

print("Sun Position & Building Shadow Functions Loaded")

Sun Position & Building Shadow Functions Loaded


### Module 3B: Enhanced Spatial Graph Engine

Objective: Build road network and overlay ALL cooling features.

**Layer 1:** Road Network (OSM)  
**Layer 2:** Park Connectors (PCN)  
**Layer 3:** Tree Canopy (SGTrees)  
**Layer 4:** Building Shadows (Time-dependent)  
**Layer 5:** Water Bodies (NEW in V5! - Proximity-based cooling)

Logic: Roads receive cumulative discounts based on shade coverage from multiple sources.

In [9]:
# ==========================================
# 🛣️ MODULE 3B: THE COOL ENGINE (V10.2 - AUTO FAILOVER)
# ==========================================
import osmnx as ox
import networkx as nx
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
import pysolar.solar as solar
from datetime import datetime

# 🛠️ SETTINGS
ox.settings.log_console = True
ox.settings.use_cache = True
ox.settings.timeout = 45 # 45s timeout per attempt

print("🛣️ LOADING COOL ENGINE (V10.2 - AUTO FAILOVER)...")

def download_graph_safe(north, south, east, west, cf):
    """
    Tries to download the map from multiple mirrors.
    """
    # LIST OF MIRRORS TO TRY
    mirrors = [
        "https://overpass-api.de/api/interpreter",   # 1. Main Server
        "https://overpass.kumi.systems/api/interpreter" # 2. Backup
    ]

    for mirror in mirrors:
        print(f"   🔄 Trying Server: {mirror}...")
        ox.settings.overpass_url = mirror
        try:
            # Universal Syntax Check
            if int(ox.__version__.split('.')[0]) >= 2:
                G = ox.graph_from_bbox(bbox=(north, south, east, west), network_type='bike', custom_filter=cf, simplify=True)
            else:
                G = ox.graph_from_bbox(north, south, east, west, network_type='bike', custom_filter=cf, simplify=True)

            print("   ✅ Download Success!")
            return G
        except Exception as e:
            print(f"   ⚠️ Failed: {e}")
            continue # Try next mirror

    print("   ❌ ALL MIRRORS FAILED.")
    return None

def generate_cool_routes():
    # 1. DEFINE BOUNDING BOX
    lats = [START_POINT[0], END_POINT[0]]
    lons = [START_POINT[1], END_POINT[1]]

    # Conservative buffer to prevent huge downloads
    lat_diff = max(lats) - min(lats)
    buffer = max(0.005, 0.01 if lat_diff < 0.05 else 0.02)

    north = max(lats) + buffer
    south = min(lats) - buffer
    east = max(lons) + buffer
    west = min(lons) - buffer

    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 2. DOWNLOAD GRAPH (With Failover)
    cf = '["highway"~"cycleway|path|living_street|residential|tertiary|secondary|primary"]'
    graph = download_graph_safe(north, south, east, west, cf)

    if graph is None: return None, None, None, None

    # 3. OVERLAY PARK CONNECTORS (PCN)
    print("⏳ Overlaying Park Connectors...")
    print("   ✅ PCN Loaded")

    # 4. LOAD TREES
    print("⏳ Loading Tree Canopy Data...")
    try:
        trees_gdf = gpd.read_file('data/Trees_SG.geojson')
        trees_gdf = trees_gdf.cx[west:east, south:north]
        if not trees_gdf.empty:
            trees_buffer = trees_gdf.geometry.buffer(0.00005).unary_union
            print(f"   ✅ Tree shade layer generated ({len(trees_gdf)} trees)")
        else:
            trees_buffer = None
    except:
        trees_buffer = None
        print("   ⚠️ Tree data missing/error.")

    # 5. LOAD BUILDINGS
    print("⏳ Loading Buildings...")
    try:
        date = DEPARTURE_TIME
        altitude = solar.get_altitude(1.3521, 103.8198, date)
        azimuth = solar.get_azimuth(1.3521, 103.8198, date)
        print(f"   ☀️ Sun (SGT): {altitude:.1f}° elev, {azimuth:.1f}° azim")

        tags = {'building': True}
        if int(ox.__version__.split('.')[0]) >= 2:
            buildings = ox.features_from_bbox(bbox=(north, south, east, west), tags=tags)
        else:
            buildings = ox.features_from_bbox(north, south, east, west, tags=tags)

        if not buildings.empty:
            shift_dist = 0.00015 * (90 - altitude) / 90
            shadow_x = -np.sin(np.radians(azimuth)) * shift_dist
            shadow_y = -np.cos(np.radians(azimuth)) * shift_dist
            shadows = buildings.translate(xoff=shadow_x, yoff=shadow_y)
            buildings_buffer = shadows.unary_union
            print("   ✅ Building shadow layer generated")
        else:
            buildings_buffer = None
    except:
        buildings_buffer = None
        print("   ⚠️ Building data missing/error.")

    # 6. WATER BODIES
    print("⏳ Loading Water Bodies...")
    try:
        water_gdf = gpd.read_file('data/URA_Waterbody.geojson')
        water_gdf = water_gdf.cx[west:east, south:north]
        if not water_gdf.empty:
            water_buffer = water_gdf.geometry.buffer(0.0005).unary_union
            print(f"   ✅ URA Water Layer Active ({len(water_gdf)} features)")
        else:
            water_buffer = None
    except:
        water_buffer = None
        print("   ⚠️ Water data missing.")

    # 7. AMENITIES
    print("⏳ Loading Amenities...")
    amenities_list = []
    try:
        tags = {'amenity': ['food_court', 'hawker_centre', 'marketplace'], 'shop': 'supermarket'}
        if int(ox.__version__.split('.')[0]) >= 2:
            pois = ox.features_from_bbox(bbox=(north, south, east, west), tags=tags)
        else:
            pois = ox.features_from_bbox(north, south, east, west, tags=tags)

        if not pois.empty:
            for idx, row in pois.iterrows():
                name = row.get('name', 'Unknown')
                if name == 'Unknown': continue
                if row.geometry.geom_type == 'Point':
                    lat, lon = row.geometry.y, row.geometry.x
                else:
                    lat, lon = row.geometry.centroid.y, row.geometry.centroid.x
                type_label = "Supermarket" if 'shop' in row and row['shop'] == 'supermarket' else "Hawker"
                amenities_list.append((name, lat, lon, type_label))
            print(f"      Found {len(amenities_list)} Amenities")
    except:
        print("      ⚠️ Amenities skipped.")

    # 8. CALCULATE COSTS
    print("⏳ Calculating Costs...")
    for u, v, k, data in graph.edges(keys=True, data=True):
        edge_len = data['length']
        cool_score = 1.0

        if 'geometry' in data:
            edge_geom = data['geometry']
        else:
            edge_geom = LineString([(graph.nodes[u]['x'], graph.nodes[u]['y']),
                                    (graph.nodes[v]['x'], graph.nodes[v]['y'])])

        if trees_buffer is not None and trees_buffer.intersects(edge_geom):
            cool_score -= 0.4
        if buildings_buffer is not None and buildings_buffer.intersects(edge_geom):
            cool_score -= 0.3
        if water_buffer is not None and water_buffer.intersects(edge_geom):
            cool_score -= 0.2

        data['cool_cost'] = edge_len * max(0.3, cool_score)

    return graph, [], [], amenities_list

🛣️ LOADING COOL ENGINE (V10.2 - AUTO FAILOVER)...


In [5]:
# ==========================================
# 🧠 MODULE 4: AI PREDICTION ENGINE (V8.0 - DIURNAL CYCLES)
# ==========================================
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

print("🧠 LOADING AI THERMAL MODEL (DIURNAL V8)...")

def predict_smart_wbgt(current_val, hour_of_day):
    """
    Predicts WBGT using a Diurnal Cycle Model (Physics-Informed AI).
    Instead of a straight line, it models the day's heat curve.
    """
    # 1. GENERATE SYNTHETIC TRAINING DATA (The "Knowledge Base")
    # We teach the AI what a "Normal Singapore Day" looks like
    hours = np.array([0, 6, 9, 12, 14, 17, 20, 23]).reshape(-1, 1)
    # Typical WBGT profile: Cool night, spike morning, peak afternoon, cool evening
    typical_profile = np.array([26.0, 25.5, 29.0, 32.5, 33.0, 31.0, 29.0, 27.0])

    # 2. FIT A POLYNOMIAL CURVE (The "Wave")
    # Degree 4 polynomial captures the double-curve of day/night
    poly = PolynomialFeatures(degree=4)
    X_poly = poly.fit_transform(hours)
    model = LinearRegression()
    model.fit(X_poly, typical_profile)

    # 3. PREDICT FOR USER'S TIME
    user_hour = np.array([[hour_of_day]])
    base_prediction = model.predict(poly.transform(user_hour))[0]

    # 4. APPLY "REALITY CORRECTION" (The "Live AI" part)
    # The model knows the 'shape' of the day, but the Sensor knows the 'actual height'.
    # If sensor says it's 35°C but model expects 33°C, we shift the whole curve up.

    # Calculate what the model *thinks* it should be right now
    # (Simplified: we assume sensor reading is 'now')
    offset = current_val - base_prediction

    # The final prediction applies this offset to the curve
    # Let's predict the heat 1 hour from now (Duration of ride)
    future_hour = np.array([[(hour_of_day + 1) % 24]])
    future_val = model.predict(poly.transform(future_hour))[0] + offset

    return future_val, offset

# Test it
# val, offset = predict_smart_wbgt(31.0, 14)
# print(f"Predicted WBGT in 1 hour: {val:.2f}°C")

🧠 LOADING AI THERMAL MODEL (DIURNAL V8)...


### 🚀 Module 5: Execution & Safe-Pace Recommendations

Objective: Synthesize map, weather, and AI data into a KML route. Upgrade:

* Govt Override: Checks NEA_HEATWAVE_ALERT.
* Safe Pacing: Suggests specific ride speeds and hydration intervals based on WBGT (ISO 7243 standards).

In [6]:
''' --> uncomment when required

# ==========================================
# 🚀 MODULE 5: EXECUTION ENGINE (V8.0 - INTEGRATED AI)
# ==========================================
import math
import os
import time
import requests
import simplekml

print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# --- HELPER: DYNAMIC SENSOR FINDER ---
def get_nearest_wbgt_station(lat, lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    try:
        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)
        data = resp.json()
        readings = data['data']['records'][0]['item'].get('readings', [])

        closest_station = "Unknown"
        min_dist = float('inf')
        current_val = None

        for r in readings:
            try:
                loc = {}
                s_name = "Unknown"
                if 'location' in r: loc = r['location']
                elif 'station' in r and 'location' in r['station']: loc = r['station']['location']
                if 'station' in r: s_name = r['station'].get('name', 'Unknown')
                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0)))
                if s_lat == 0 or s_lon == 0: continue

                val = r.get('wbgt') or r.get('value')
                if val is None: continue
                val = float(val)
                dist = math.sqrt((lat - s_lat)**2 + (lon - s_lon)**2)

                if dist < min_dist:
                    min_dist = dist
                    closest_station = s_name
                    current_val = val
            except: continue

        if current_val is None: return 30.0, "System Fallback"
        print(f"   📍 Nearest Sensor: {closest_station} (Dist: {min_dist*111:.2f} km)")
        return current_val, closest_station
    except Exception as e:
        print(f"   ⚠️ WBGT Sensor Fail: {e}. Using Default Safety Value.")
        return 30.0, "System Fallback"

# 1. GENERATE ROUTES
graph, r1, r2, amenities = generate_cool_routes()

if graph:
    # 2. GET WEATHER (LIVE SENSOR)
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. RUN AI (UPDATED: DIURNAL MODEL)
    # Replaced 'predict_trend' with 'predict_smart_wbgt'
    ride_hour = DEPARTURE_TIME.hour
    forecast_wbgt, heat_offset = predict_smart_wbgt(current_wbgt, ride_hour)

    # Calculate effective risk
    effective_wbgt = max(current_wbgt, forecast_wbgt)
    if NEA_HEATWAVE_ALERT: effective_wbgt = 35.0

    # 4. REPORT
    if effective_wbgt < 29: rec = "✅ Safe to Ride."
    elif effective_wbgt < 31: rec = "⚠️ CAUTION: Seek shade."
    else: rec = "🛑 HIGH RISK: Stop."

    print(f"\n📊 REPORT: {station_name}")
    print(f"   Current: {current_wbgt:.1f}°C | Forecast (1h): {forecast_wbgt:.1f}°C")
    print(f"   🤖 AI Insight: Offset is {heat_offset:+.1f}°C from historical average.")

    # 5. EXPORT KML
    kml = simplekml.Kml()

    def add_route(route, color, name, description):
        ls = kml.newlinestring(name=name)
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            d = graph.get_edge_data(u, v)[0]
            if 'geometry' in d:
                xs, ys = d['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        ls.coords = coords
        ls.style.linestyle.color = color
        ls.style.linestyle.width = 5
        ls.description = description

    # 🧠 FUZZY LOGIC
    def check_similarity(route_a, route_b):
        set_a = set(route_a)
        set_b = set(route_b)
        intersection = len(set_a.intersection(set_b))
        union = len(set_a.union(set_b))
        return intersection / union

    sim_score = check_similarity(r1, r2)
    print(f"   🔍 Route Similarity Score: {sim_score*100:.1f}%")

    if sim_score > 0.90:
        print("   💡 Insight: Routes are effectively identical (Merged).")
        add_route(r2, simplekml.Color.green, "🌟 Recommended Route",
                  f"<b>Smart Choice</b><br>The fastest path is also the coolest.<br>Temp: {effective_wbgt:.1f}°C")
    else:
        print("   💡 Insight: A distinct cooler detour exists.")
        add_route(r1, simplekml.Color.red, "⚡ Fastest Route (Exposed)",
                  f"<b>Direct Path</b><br>Shortest time, but higher heat exposure.<br>Temp: {effective_wbgt:.1f}°C")
        add_route(r2, simplekml.Color.green, "🌿 Cool Route (Shaded)",
                  f"<b>Shaded Detour</b><br>Maximized tree canopy coverage.<br>Lower heat stress.<br>Temp: {effective_wbgt:.1f}°C")

    # --- AMENITIES SECTION ---
    from shapely.geometry import Point, LineString

    cool_route_coords = []
    for u, v in zip(r2[:-1], r2[1:]):
        d = graph.get_edge_data(u, v)[0]
        if 'geometry' in d:
            xs, ys = d['geometry'].xy
            cool_route_coords.extend(list(zip(xs, ys)))
        else:
            cool_route_coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
            cool_route_coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))

    route_geom = LineString(cool_route_coords)

    count_amenities = 0
    if amenities:
        for name, lat, lon, type_label in amenities:
            poi_point = Point(lon, lat)
            if route_geom.distance(poi_point) < 0.003:
                p = kml.newpoint(name=f"{type_label}: {name}", coords=[(lon, lat)])

                if type_label == "Hawker":
                    p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/dining.png'
                    p.description = "<b>Hawker Centre</b><br>Cheap food & shelter."
                else:
                    p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/grocery.png'
                    p.description = "<b>Supermarket</b><br>Water & supplies."

                count_amenities += 1

    print(f"   🍱 Added {count_amenities} amenities near the route.")

    # 6. SAVE
    if not os.path.exists('output'): os.makedirs('output')
    constant_filename = "output/latest_route.kml"
    kml.save(constant_filename)

    print(f"\n🎉 SUCCESS! Download '{constant_filename}'")
else:
    print("❌ Critical Error: Route Generation Failed.")

'''

' --> uncomment when required\n\n# ==========================================\n# 🚀 MODULE 5: EXECUTION ENGINE (V8.0 - INTEGRATED AI)\n# ==========================================\nimport math\nimport os\nimport time\nimport requests\nimport simplekml\n\nprint("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")\n\n# --- HELPER: DYNAMIC SENSOR FINDER ---\ndef get_nearest_wbgt_station(lat, lon):\n    print("⏳ Connecting to NEA Official WBGT Sensor Network...")\n    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"\n    try:\n        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)\n        data = resp.json()\n        readings = data[\'data\'][\'records\'][0][\'item\'].get(\'readings\', [])\n\n        closest_station = "Unknown"\n        min_dist = float(\'inf\')\n        current_val = None\n\n        for r in readings:\n            try:\n                loc = {}\n                s_name = "Unknown"\n                if \'location\' in r: loc = r[\'location\']\n        

In [7]:
''' --> Uncomment when required

# ==========================================
# 📤 MODULE 6: CLOUD SYNC (V6.1 - SMART FIX)
# ==========================================
def push_to_github(filename):
    print("\n☁️ INITIATING CLOUD SYNC...")

    # 1. RETRIEVE TOKEN
    try:
        from google.colab import userdata
        token = userdata.get('GITHUB_TOKEN')
    except:
        token = input("   Enter GitHub PAT (Token): ")

    if not token:
        print("   ❌ Sync Failed: No Token provided.")
        return

    # 2. SETUP
    repo_url = f"https://{token}@github.com/{GITHUB_USER}/{REPO_NAME}.git"
    user_email = "coolride.bot@gmail.com"
    user_name = "CoolRide Bot"

    # 3. SMART COMMANDS (Auto-detects branch + Creates folder)
    import subprocess

    # Helper to run commands safely
    def run_git(cmd):
        try:
            # We filter out the token from errors so it doesn't leak in logs
            result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
            return True, result.stdout
        except subprocess.CalledProcessError as e:
            error_msg = e.stderr.replace(token, "HIDDEN_TOKEN") # Safety mask
            print(f"   ⚠️ Git Error: {error_msg}")
            return False, error_msg

    print("   ⏳ Cloning repository...")
    run_git(f"rm -rf temp_repo") # Clean start
    success, _ = run_git(f"git clone {repo_url} temp_repo")

    if not success: return

    print("   ⏳ Processing files...")
    # Smart Move: Ensure folder exists, Copy file, Configure Git, Push to HEAD (Current Branch)
    commands = [
        f"mkdir -p temp_repo/output", # Force create folder
        f"cp {filename} temp_repo/output/latest_route.kml",
        f"cp {filename} temp_repo/output/{os.path.basename(filename)}",
        f"cd temp_repo && git config user.email '{user_email}'",
        f"cd temp_repo && git config user.name '{user_name}'",
        "cd temp_repo && git add .",
        f"cd temp_repo && git commit -m '🤖 AI Update: {time.strftime('%H:%M')}'",
        "cd temp_repo && git push origin HEAD" # Pushes to whatever branch (main/master) is active
    ]

    for cmd in commands:
        success, _ = run_git(cmd)
        if not success:
            print("   ❌ Sync Aborted due to error above.")
            return

    print("   ✅ CLOUD SYNC COMPLETE.")
    print(f"   🌐 View: https://github.com/{GITHUB_USER}/{REPO_NAME}/blob/master/output/latest_route.kml")

# EXECUTE
if 'constant_filename' in locals():
    push_to_github(constant_filename)

'''

' --> Uncomment when required\n\n# ==========================================\n# 📤 MODULE 6: CLOUD SYNC (V6.1 - SMART FIX)\n# ==========================================\ndef push_to_github(filename):\n    print("\n☁️ INITIATING CLOUD SYNC...")\n\n    # 1. RETRIEVE TOKEN\n    try:\n        from google.colab import userdata\n        token = userdata.get(\'GITHUB_TOKEN\')\n    except:\n        token = input("   Enter GitHub PAT (Token): ")\n\n    if not token:\n        print("   ❌ Sync Failed: No Token provided.")\n        return\n\n    # 2. SETUP\n    repo_url = f"https://{token}@github.com/{GITHUB_USER}/{REPO_NAME}.git"\n    user_email = "coolride.bot@gmail.com"\n    user_name = "CoolRide Bot"\n\n    # 3. SMART COMMANDS (Auto-detects branch + Creates folder)\n    import subprocess\n\n    # Helper to run commands safely\n    def run_git(cmd):\n        try:\n            # We filter out the token from errors so it doesn\'t leak in logs\n            result = subprocess.run(cmd, shell=True, 

# Live Engine Module (Module 7)

In [8]:
# ==========================================
# 🌐 MODULE 7: LIVE API SERVER (V9.7 - MARKERS & TOGGLES)
# ==========================================
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import osmnx as ox
import networkx as nx
import pytz
from datetime import datetime
import simplekml
import time
from shapely.geometry import Point, LineString

# ---------------------------------------------------------
# 🔑 AUTHENTICATION
# ---------------------------------------------------------
ngrok.set_auth_token("36uk5sD1Xy2OufMem31eLZ9tXLh_7zWriMUTYH9WUGNTgciyG")
# ---------------------------------------------------------

app = Flask(__name__)
CORS(app)

@app.route('/calculate_route', methods=['POST'])
def handle_route_request():
    start_timer = time.time()
    global START_POINT, END_POINT, DEPARTURE_TIME, PLACE_NAME

    try:
        data = request.json
        print(f"\n📨 NEW REQUEST: {data}")

        start_text = data.get('start', 'Tampines MRT')
        end_text = data.get('end', 'Tampines Eco Green')
        stop_text = data.get('stop', '')
        time_text = data.get('time', '')

        PLACE_NAME = f"{start_text} to {end_text}"

        # 1. Geocode
        try:
            START_POINT = ox.geocode(start_text + ", Singapore")
            END_POINT = ox.geocode(end_text + ", Singapore")
            STOP_POINT = None
            if stop_text:
                if "," in stop_text:
                    lat, lon = map(float, stop_text.split(","))
                    STOP_POINT = (lat, lon)
                else:
                    STOP_POINT = ox.geocode(stop_text + ", Singapore")
        except:
            return jsonify({"status": "error", "message": "Location not found."}), 400

        # 2. Time Handling
        sgt_zone = pytz.timezone('Asia/Singapore')
        if not time_text:
            DEPARTURE_TIME = datetime.now(sgt_zone)
        else:
            try:
                hour, minute = map(int, time_text.split(':'))
                DEPARTURE_TIME = datetime.now(sgt_zone).replace(hour=hour, minute=minute, second=0)
            except:
                DEPARTURE_TIME = datetime.now(sgt_zone)

        # 3. Engine
        print(f"   ⚙️ Running AI Engine for: {PLACE_NAME}...")
        graph, _, _, amenities = generate_cool_routes()
        if graph is None: return jsonify({"status": "error", "message": "Graph failed."}), 500

        # Solver Helper
        def solve_path(start_node, end_node):
            try:
                r_fast = nx.shortest_path(graph, start_node, end_node, weight='length')
                r_cool = nx.shortest_path(graph, start_node, end_node, weight='cool_cost')
                return r_fast, r_cool
            except:
                return [], []

        orig_node = ox.distance.nearest_nodes(graph, START_POINT[1], START_POINT[0])
        dest_node = ox.distance.nearest_nodes(graph, END_POINT[1], END_POINT[0])

        # 4. PATH SOLVING
        if STOP_POINT:
            stop_node = ox.distance.nearest_nodes(graph, STOP_POINT[1], STOP_POINT[0])
            l1_fast, l1_cool = solve_path(orig_node, stop_node)
            l2_fast, l2_cool = solve_path(stop_node, dest_node)
            if l1_fast and l2_fast:
                r1 = l1_fast[:-1] + l2_fast
                r2 = l1_cool[:-1] + l2_cool
            else:
                r1, r2 = [], []
        else:
            r1, r2 = solve_path(orig_node, dest_node)

        if not r1: return jsonify({"status": "error", "message": "No path found."}), 500

        # --- SMART ANALYTICS ---
        def analyze_route(route):
            total_len = 0
            shaded_len = 0
            for u, v in zip(route[:-1], route[1:]):
                data = graph.get_edge_data(u, v)[0]
                length = data['length']
                total_len += length
                if data['cool_cost'] < (length * 0.9):
                    shaded_len += length

            eta_mins = int(total_len / 250)
            shade_pct = int((shaded_len / total_len) * 100) if total_len > 0 else 0
            return eta_mins, shade_pct

        fast_eta, fast_shade = analyze_route(r1)
        cool_eta, cool_shade = analyze_route(r2)
        shade_gain = cool_shade - fast_shade

        # --- AI FORECAST ---
        current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])
        ride_hour = DEPARTURE_TIME.hour
        forecast_wbgt, heat_offset = predict_smart_wbgt(current_wbgt, ride_hour)

        if shade_gain > 20:
            insight = f"<b>Smart Choice:</b> The Cool Route provides <b>{shade_gain}% more shade coverage</b>, effectively reducing solar heat load by ~{shade_gain/10:.1f}°C."
            status_color = "green"
        elif heat_offset > 1.0:
            insight = f"<b>Heat Alert:</b> It is {heat_offset:.1f}°C hotter than average. Avoid the exposed Fastest Route."
            status_color = "red"
        else:
            insight = "Conditions are mild. Both routes are acceptable, but Green is more scenic."
            status_color = "green"

        # 5. Generate KML
        kml = simplekml.Kml()

        def add_kml_line(route, color, name, desc):
            ls = kml.newlinestring(name=name)
            coords = []
            for u, v in zip(route[:-1], route[1:]):
                d = graph.get_edge_data(u, v)[0]
                if 'geometry' in d:
                    xs, ys = d['geometry'].xy
                    coords.extend([(float(x), float(y)) for x, y in zip(xs, ys)])
                else:
                    coords.append((float(graph.nodes[u]['x']), float(graph.nodes[u]['y'])))
                    coords.append((float(graph.nodes[v]['x']), float(graph.nodes[v]['y'])))
            ls.coords = coords
            ls.style.linestyle.color = color
            ls.style.linestyle.width = 5
            ls.description = desc
            return ls, coords

        desc_fast = f"""
        <div style='font-family:sans-serif; width:200px;'>
            <h3 style='margin:0;color:#ef4444;'>⚡ Fastest Route</h3>
            <p style='margin:5px 0;'><b>{fast_eta} mins</b> • {fast_shade}% Shaded</p>
            <p style='font-size:11px;color:#64748b;'>High solar exposure. Not recommended during midday.</p>
        </div>
        """

        desc_cool = f"""
        <div style='font-family:sans-serif; width:200px;'>
            <h3 style='margin:0;color:#22c55e;'>🌿 Cool Route</h3>
            <p style='margin:5px 0;'><b>{cool_eta} mins</b> • {cool_shade}% Shaded</p>
            <p style='font-size:11px;color:#64748b;'>Optimized for tree canopy & building shadows.</p>
        </div>
        """

        add_kml_line(r1, simplekml.Color.red, "Fastest Route", desc_fast)
        _, cool_coords = add_kml_line(r2, simplekml.Color.green, "Cool Route", desc_cool)

        # 6. Amenities
        route_geom = LineString(cool_coords)
        count = 0
        for name, lat, lon, type_label in amenities:
            lat, lon = float(lat), float(lon)
            poi_point = Point(lon, lat)
            if route_geom.distance(poi_point) < 0.003:
                emoji = "🍜" if type_label == "Hawker" else "🛒"
                p = kml.newpoint(name=f"{emoji} {name}")
                p.coords = [(lon, lat)]
                safe_name = name.replace("'", "").replace('"', "")
                p.description = f"""
                <b>{type_label}</b><br>{name}<br><br>
                <button onclick="parent.addStop('{lat},{lon}', '{safe_name}')"
                style="background:#2563eb;color:white;border:none;padding:5px 10px;cursor:pointer;border-radius:4px;">
                ➕ Add Stop Here
                </button>
                """
                count += 1

        duration = time.time() - start_timer
        print(f"⏱️ Generation Time: {duration:.2f}s")

        return jsonify({
            "status": "success",
            "kml_data": kml.kml(),
            "meta": {
                "duration": f"{duration:.2f}s",
                "start_point": START_POINT, # SEND START COORDS
                "end_point": END_POINT      # SEND END COORDS
            },
            "ai_data": {
                "current_temp": f"{current_wbgt:.1f}",
                "forecast_temp": f"{forecast_wbgt:.1f}",
                "insight": insight,
                "color": status_color,
                "shade_gain": shade_gain
            }
        })

    except Exception as e:
        print(f"Server Error Traceback: {e}")
        return jsonify({"status": "error", "message": str(e)}), 500

public_url = ngrok.connect(5000).public_url
print(f"🚀 SERVER ONLINE! API URL: {public_url}")
app.run(port=5000)

🚀 SERVER ONLINE! API URL: https://neurally-submucronate-sonny.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 12:45:45] "OPTIONS /calculate_route HTTP/1.1" 200 -



📨 NEW REQUEST: {'start': 'Tampines MRT', 'end': 'Tampines Eco Green', 'time': '14:00', 'stop': ''}
   ⚙️ Running AI Engine for: Tampines MRT to Tampines Eco Green...
⏳ Downloading road network for Tampines MRT to Tampines Eco Green...
   🌍 Source: OpenStreetMap.fr (Backup Mirror)
   📐 Zone: Lat[1.3430, 1.3737], Lon[103.9354, 103.9582]


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 12:46:51] "POST /calculate_route HTTP/1.1" 500 -


   ⚠️ Primary Download Failed: HTTPSConnectionPool(host='api.openstreetmap.fr', port=443): Max retries exceeded with url: /oapi/interpreter/interpreter (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78cbc526bef0>: Failed to resolve 'api.openstreetmap.fr' ([Errno -2] Name or service not known)"))
